In [ ]:
from scipy.integrate import odeint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objs as go
%matplotlib inline

In [ ]:
def deriv(y, t, N, beta, gamma, delta):
    S, E, I, R = y
    dSdt = -beta * S * I / N
    dEdt = beta * S * I / N - delta * E
    dIdt = delta * E - gamma * I
    dRdt = gamma * I
    return dSdt, dEdt, dIdt, dRdt

In [ ]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.Img(id='corona_png',src='assets/covid-19.png'),
    html.Img(id='um6p_png',src='assets/um6p.png'),
    html.Div([
        html.Div([
            html.Label(children='Nomber of Susceptibles :'),
            dcc.Input(
                id='susceptibles',
                type='number',
                value=999
            ),

            html.Label(children='Nomber des infecteds :'),
            dcc.Input(
                id='infecteds',
                type='number',
                value=1
            ),

            html.Label(children='Recovered :'),
            dcc.Input(
                id = 'recovered',
                type='number',
                value=0
            ),

            html.Label(children='Days :'),
            dcc.Input(
                id = 'days',
                type='number',
                value=100
            ),
            
            html.Label(children='Number of susceptibles could be infect by an infected (Probabilty of an susceptible becomes exposed):'),
            dcc.Slider(
                id='exposed',
                min=0,
                max=10,
                value=1,
                marks = {str(round(val,2)) : str(round(val,2)) for val in np.arange(0,11)},
                step=None
            ),

            html.Label(children='Number of exposeds become infected (Probabilty of an exposeds becomes infected):'),
            dcc.Slider(
                id='infected',
                min=0,
                max=10,
                value=1,
                marks = {str(round(val,2)) : str(round(val,2)) for val in np.arange(0,11)},
                step=None
            ),


            html.Label(children='period of infections:'),
            dcc.Slider(
                id='infection',
                min=0,
                max=10,
                value=4,
                marks = {str(round(val,2)) : str(round(val,2)) for val in np.arange(0,11)},
                step=None
            ),


            html.Label(children='period of incubation:'),
            dcc.Slider(
                id='incubation',
                min=0,
                max=10,
                value=3,
                marks = {str(round(val,2)) : str(round(val,2)) for val in np.arange(0,11)},
                step=None
            ),
            
            html.Button(id='submit-button', n_clicks=0, children='Submit')

        ],id='form'),

        dcc.Graph(id='indicator-graphic'),
        #dcc.Graph(id='scatter-graphic'),
    ],id="resultat")
    
],className = 'container')


@app.callback(
    Output('indicator-graphic', 'figure'),
    [Input('submit-button', 'n_clicks')],
    [State('susceptibles', 'value'),
    State('infecteds', 'value'),
    State('recovered', 'value'),
    State('days', 'value'),
    State('exposed', 'value'),
    State('infected', 'value'),
    State('infection', 'value'),
    State('incubation', 'value')]
)
def update_figure(n_clicks,susceptibles,infecteds,recovered,days,exposed,infected,infection,incubation):
    N = susceptibles+infecteds+infecteds
    beta = exposed
    D = infection
    gamma = infected / D
    delta = infected / incubation  #à verfier

    S0, E0, I0, R0 = susceptibles, 0, infecteds, 0  
    t = np.linspace(0, days, N)

    y0 = S0, E0, I0, R0 

    # Integrate the SIR equations over the time grid, t.
    ret = odeint(deriv, y0, t, args=(N, beta, gamma, delta))
    S, E, I, R = ret.T
    
    
    traces = [
            go.Scatter(x=[0, 0], y=[0, 0], name='S'),
            go.Scatter(x=[0, 0], y=[0, 0], name='E'),
            go.Scatter(x=[0, 0], y=[0, 0], name='I'),
            go.Scatter(x=[0, 0], y=[0, 0], name='R')
    ]
        
    layout = go.Layout(
        xaxis=dict(range=[-5, 110], autorange=False),
        yaxis=dict(range=[0, 10], autorange=False),
        title="SEIR Model",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    )
    frames = [go.Frame(data=[go.Scatter(x=np.sort(np.random.choice(t,int(S[i]))), y=np.random.normal(5.0, 1.0, int(S[i])), name='S', mode='markers'),
                           go.Scatter(x=np.sort(np.random.choice(t,int(E[i]))), y=np.random.normal(5.0, 1.0, int(E[i])), name='E', mode='markers'),
                           go.Scatter(x=np.sort(np.random.choice(t,int(I[i]))), y=np.random.normal(5.0, 1.0, int(I[i])), name='I', mode='markers'),
                           go.Scatter(x=np.sort(np.random.choice(t,int(R[i]))), y=np.random.normal(5.0, 1.0, int(R[i])), name='R', mode='markers')]
                    )for i in range(N)]
    
    return {
        'data' : traces,
        'layout' : layout,
        'frames' :frames
    }

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=False)

directement par plotly, utilisation de Dash consomme vraiment le memoire.

In [ ]:
N = 1000
beta = 1.0  # infected person infects 1 other person per day
D = 4.0 # infections lasts four days
gamma = 1.0 / D
delta = 1.0 / 3.0  # incubation period of three days

S0, E0, I0, R0 = 999, 1, 0, 0  # initial conditions: one exposed, rest susceptible

In [ ]:
t = np.linspace(0, 100, 100) # Grid of time points (in days)
y0 = S0, E0, I0, R0 # Initial conditions vector

# Integrate the SIR equations over the time grid, t.
ret = odeint(deriv, y0, t, args=(N, beta, gamma, delta))
S, E, I, R = ret.T

In [ ]:
t = np.linspace(0, 100,1000)
fig = go.Figure(
    data=[go.Scatter(x=[0, 0], y=[0, 0],name='S', mode='markers'),
         go.Scatter(x=[0, 0], y=[0, 0],name='E', mode='markers'),
         go.Scatter(x=[0, 0], y=[0, 0],name='I', mode='markers'),
         go.Scatter(x=[0, 0], y=[0, 0],name='R', mode='markers')],
    layout=go.Layout(
        #xaxis=dict(range=[0, 100], autorange=False),
        #yaxis=dict(range=[0, 10], autorange=False),
        title="SEIR Model",
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          args=[None])])]
    ),
    frames=[go.Frame(data=[go.Scatter(x=np.sort(np.random.choice(t,int(S[i]))), y=np.random.normal(5.0, 1.0, int(S[i])),name='S'),
                           go.Scatter(x=np.sort(np.random.choice(t,int(E[i]))), y=np.random.normal(5.0, 1.0, int(E[i])),name='E'),
                           go.Scatter(x=np.sort(np.random.choice(t,int(I[i]))), y=np.random.normal(5.0, 1.0, int(I[i])),name='I'),
                           go.Scatter(x=np.sort(np.random.choice(t,int(R[i]))), y=np.random.normal(5.0, 1.0, int(R[i])),name='R')]
                    )for i in range(100)]
)

fig.show()